# Production Planning
Slide 40

### N Possible Values

In [1]:
import pandas as pd
import numpy as np

import gurobipy as gb
from gurobipy import *
from gurobipy import Model, GRB, quicksum as sum

In [2]:
product = ['A1', 'A2', 'A3', 'A4']
y_days = ['Y1', 'Y2', 'Y3', 'Y4']
profit = [62, 84, 103, 125]
upper = [8, 17, 11, 15]
days = [270, 300, 345, 360]
p = len(product)
y = len(y_days)

In [3]:
prob = gb.Model("Production Planning")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-30


In [4]:
X = prob.addVar(lb = 0, vtype = GRB.INTEGER, name = "X")

In [5]:
Y = prob.addVars(y, lb = 0, ub = 1, vtype = GRB.BINARY, name = y_days)

In [6]:
A = prob.addVars(p, lb = 0, ub=upper, vtype = GRB.INTEGER, name = product)

In [7]:
prob.setObjective(sum(profit[i]*A[i] - (X - 270) for i in range(p))*1000, GRB.MAXIMIZE)

In [8]:
prob.addConstr(sum(days[i]*Y[i] for i in range(p)) == X, name = "Total manufacturing days") 

<gurobi.Constr *Awaiting Model Update*>

In [9]:
prob.addConstr(sum(Y[i] for i in range(p)) == 1, name = "Additional days option") 

<gurobi.Constr *Awaiting Model Update*>

In [10]:
prob.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2 rows, 9 columns and 9 nonzeros
Model fingerprint: 0x8a2a8e34
Variable types: 0 continuous, 9 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [4e+03, 1e+05]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 1e+00]
Presolve removed 2 rows and 9 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 4.932e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.932000000000e+06, best bound 4.932000000000e+06, gap 0.0000%


In [11]:
for v in prob.getVars():    
    print(v.varName, "=", round(v.X))
print(f"Optimal Objective Value: {prob.ObjVal}")

X = 270
Y1 = 1
Y2 = 0
Y3 = 0
Y4 = 0
A1 = 8
A2 = 17
A3 = 11
A4 = 15
Optimal Objective Value: 4932000.0


In class notes
- N Possible values (Discrete Optimization Page 41)
- add X
- add yi
- edit objective function: same function -(X-270)*1000
- X = 270y1+...
- sum(yi)=1
- X= 270, y1=1, y2...y4=0, The optimal soultion won't be changing as the answer is the same.

### The Fixed Charge Problem

In [12]:
import pandas as pd
import numpy as np

import gurobipy as gb
from gurobipy import *
from gurobipy import Model, GRB, quicksum as sum

In [13]:
prob = gb.Model("Fixed Charge Problem")

In [14]:
A = prob.addVars(p, lb=0, ub=upper, vtype=GRB.INTEGER, name = product)

In [15]:
X = prob.addVar(lb=0, vtype=GRB.INTEGER, name="X") # X = the increase in the number of days

In [16]:
Y = prob.addVar(lb=0, ub=1, vtype=GRB.BINARY, name = "Y")

In [17]:
# $2500 fixed cost, charges $1000 per manufacturing days
prob.setObjective(sum(profit[i]*A[i]-(2500*Y + 1000*X) for i in range(p))*1000, GRB.MAXIMIZE)

In [18]:
manu_days = [4, 7, 9, 11]
prob.addConstr(sum(manu_days[i]*A[i] for i in range(p)) <= 270 + X, name = "Manufacturing Days Constraint 1") 

# adding 270 + X -> bc if y =1, M is a large number, so X might be really large? So we need a constraint?!

<gurobi.Constr *Awaiting Model Update*>

In [19]:
M = 1000000  # M is a large number
prob.addConstr(X <= M * Y, name = "Manufacturing Days Constraint 2") 

<gurobi.Constr *Awaiting Model Update*>

In [20]:
prob.setObjective(sum(profit[i]*A[i] - (2500*Y + 1000*X) for i in range(p)), GRB.MAXIMIZE)

In [21]:
prob.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2 rows, 6 columns and 7 nonzeros
Model fingerprint: 0x02ea2d5e
Variable types: 0 continuous, 6 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [6e+01, 1e+04]
  Bounds range     [1e+00, 2e+01]
  RHS range        [3e+02, 3e+02]
Found heuristic solution: objective 3182.0000000
Presolve removed 2 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 2: 3277 3182 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.277000000000e+03, best bound 3.277000000000e+03, gap 0.0000%


In [22]:
for v in prob.getVars():    
    print(v.varName, "=", round(v.X))
print(f"Optimal Objective Value: {prob.ObjVal}")

A1 = 8
A2 = 17
A3 = 1
A4 = 10
X = 0
Y = 0
Optimal Objective Value: 3277.0


- X, Y
- old object function -(2500y + 1000X)
- Constrs. 
- 4A1 + 7A2 + 9A3 + 11A4 <= 270+ X
- X <= M*Y
- Result: X=0. I do not buy any manufacturing days.
- This problem: how can we capture fixed cost? And the above probem is about ho to capture the additional manufacturing days